# SQLite database

In [1]:
import sqlite3
import pandas as pd
import os.path

Create Database structure

In [4]:
def init_db():
    con = sqlite3.connect('contrcalc.db')
    cur = con.cursor()
    cur.execute('''
    CREATE TABLE IF NOT EXISTS price_table(
        price_id INTEGER PRIMARY KEY,
        commodity_id INTEGER NOT NULL,
        date_fullfillment DATE NOT NULL,
        date_price DATE NOT NULL,
        price REAL NOT NULL,
        currency TEXT NOT NULL
        );
    ''')    
    con.commit() 
    cur.execute('''    
    CREATE TABLE IF NOT EXISTS price_commodities(
        price_id INTEGER,
        commodity_id INTEGER,
        PRIMARY KEY (price_id),
        FOREIGN KEY (price_id)
            REFERENCES price_table (price_id)
                ON DELETE CASCADE
                ON UPDATE NO ACTION,
        FOREIGN KEY (commodity_id)
            REFERENCES commodities (commodity_id)
                ON DELETE CASCADE
                ON UPDATE NO ACTION
        );  
    ''')    
    
    cur.execute('''
    CREATE TABLE IF NOT EXISTS contracts
        (contract_id INTEGER PRIMARY KEY UNIQUE,
        commodity_id INTEGER NOT NULL,
        price_per_to REAL NOT NULL,
        amount_to REAL NOT NULL,
        date_closure DATE NOT NULL,
        date_fullfillment DATE NOT NULL
        );
    ''')
    con.commit()  
    
    cur.execute('''
    CREATE TABLE IF NOT EXISTS contracts_commodities(
        contract_id INTEGER,
        commodity_id INTEGER,
        PRIMARY KEY (contract_id, commodity_id),
        FOREIGN KEY (contract_id)
            REFERENCES contracts (contract_id)
                ON DELETE CASCADE
                ON UPDATE NO ACTION,
        FOREIGN KEY (commodity_id)
            REFERENCES commodities (commodity_id)
                ON DELETE CASCADE
                ON UPDATE NO ACTION
    );
    ''')
    con.commit()
    
    cur.execute('''
    CREATE TABLE IF NOT EXISTS commodities(
        commodity_id INTEGER PRIMARY KEY UNIQUE,
        name TEXT UNIQUE NOT NULL,
        harvest_month INTEGER NOT NULL,
        sowing_month INTEGER NOT NULL,
        reference_harvest_to REAL NOT NULL,
        estimate_harvest_to REAL
    );
    
    ''')
    con.commit()
    con.close()
    return

In [5]:
init_db()

Show DATABASE Structures:

In [6]:
def show_db_tables():
    con = sqlite3.connect('contrcalc.db')
    cursor = con.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    table_list = cursor.fetchall()
    con.close()
    return table_list

In [7]:
table_list = show_db_tables()
table_list

[('price_table',),
 ('price_commodities',),
 ('contracts',),
 ('contracts_commodities',),
 ('commodities',)]

Populate Commodities Table

In [11]:
def populate_commodities():
    commodities = [ 'Barley', 'Wheat', 'Corn', 'Rapeseed']
    harvest_month= [6, 8, 9, 7]
    sowing_month= [8, 9, 4, 8]
    reference_harvest_to = [7.88, 6.4, 8.2, 3.69]
    con = sqlite3.connect('contrcalc.db')
    cursor = con.cursor()
    for i in range(0, len(commodities)):
        cursor.execute("""
    INSERT INTO commodities (name, harvest_month, sowing_month, reference_harvest_to)
        VALUES(?,?,?,?)
        """, (commodities[i], harvest_month[i], sowing_month[i], reference_harvest_to[i]))
    con.commit()
    con.close()
    return
populate_commodities()

In [9]:
def get_table_df(table_name, database_path='contrcalc.db'):
    """Returns (table, connection). table is a pandas DataFrame."""
    #BASE_DIR = os.path.dirname(os.path.abspath(__file__))
    #db_path = os.path.join(BASE_DIR, database_path)
    con = sqlite3.connect(database_path)
    try:
        df = pd.read_sql("SELECT * FROM %s" %table_name, con)
        #  print("\nLoading %s table from SQLite3 database." % table_name)
    except DatabaseError as e:
        if 'no such table' in e.args[0]:
            print("\nNo such table: %s" % table_name)
            print("Create the table before loading it. " +
                  "Consider using the create_sqlite_table function")
            raise DatabaseError
        else:
            print(e)
            raise Exception("Failed to create %s table. Unknown error." %
                            table_name)
    return df, con

In [10]:
con = sqlite3.connect('contrcalc.db')
con.execute("""
ALTER TABLE price_table ADD COLUMN date_fullfillment1 DATE NOT NULL
""")
con.execute("""
ALTER TABLE price_table ADD COLUMN date_fullfillment1 DATE NOT NULL
""")
con.commit()

In [12]:
commodities_df = get_table_df('commodities')
commodities_df

(   commodity_id      name  harvest_month  sowing_month  reference_harvest_to  \
 0             1    Barley              6             8                  7.88   
 1             2     Wheat              8             9                  6.40   
 2             3      Corn              9             4                  8.20   
 3             4  Rapeseed              7             8                  3.69   
 
   estimate_harvest_to  
 0                None  
 1                None  
 2                None  
 3                None  ,
 <sqlite3.Connection at 0x220288fa990>)

In [13]:
con = sqlite3.connect('contrcalc.db')
pd.read_sql('price_table', con)

DatabaseError: Execution failed on sql 'price_table': near "price_table": syntax error